In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basic Spark Operations').getOrCreate()

In [2]:
spark

# Create DataFrame From Scratch

In [10]:
from pyspark.sql import Row
from pyspark.sql.types import StringType,StructField,StructType,LongType

In [12]:
schema = StructType([
    StructField(name='city',dataType=StringType(),nullable=True),
    StructField(name='country',dataType=StringType(),nullable=True),
    StructField(name='counts',dataType=LongType(),nullable=True),
])

In [13]:
rows = [
    Row('Los Angeles','United States',6),
    Row('New York','United States',4),
    Row('London','United Kingdom',2),
]

In [14]:
parallelizeRows = spark.sparkContext.parallelize(rows)

In [15]:
type(parallelizeRows)

pyspark.rdd.RDD

In [16]:
df = spark.createDataFrame(parallelizeRows,schema)

In [17]:
df.show()

+-----------+--------------+------+
|       city|       country|counts|
+-----------+--------------+------+
|Los Angeles| United States|     6|
|   New York| United States|     4|
|     London|United Kingdom|     2|
+-----------+--------------+------+



## different files to DataFrame

In [18]:
df_csv = spark.read.csv(r'C:\Users\admin\Desktop\spark-data\fakefriends.csv',inferSchema=True,header=True) 

In [19]:
df_csv.show()

+---+--------+---+---+
|  0|    Will| 33|385|
+---+--------+---+---+
|  1|Jean-Luc| 26|  2|
|  2|    Hugh| 55|221|
|  3|  Deanna| 40|465|
|  4|   Quark| 68| 21|
|  5|  Weyoun| 59|318|
|  6|  Gowron| 37|220|
|  7|    Will| 54|307|
|  8|  Jadzia| 38|380|
|  9|    Hugh| 27|181|
| 10|     Odo| 53|191|
| 11|     Ben| 57|372|
| 12|   Keiko| 54|253|
| 13|Jean-Luc| 56|444|
| 14|    Hugh| 43| 49|
| 15|     Rom| 36| 49|
| 16|  Weyoun| 22|323|
| 17|     Odo| 35| 13|
| 18|Jean-Luc| 45|455|
| 19|  Geordi| 60|246|
| 20|     Odo| 67|220|
+---+--------+---+---+
only showing top 20 rows



In [22]:
df_json = spark.read.json(r'C:\Users\admin\Desktop\json_data\orders.json')
df_json.show()

+-----------------+--------------------+--------+---------------+
|order_customer_id|          order_date|order_id|   order_status|
+-----------------+--------------------+--------+---------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|
|            12111|2013-07-25 00:00:...|       3|       COMPLETE|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|
|            11318|2013-07-25 00:00:...|       5|       COMPLETE|
|             7130|2013-07-25 00:00:...|       6|       COMPLETE|
|             4530|2013-07-25 00:00:...|       7|       COMPLETE|
|             2911|2013-07-25 00:00:...|       8|     PROCESSING|
|             5657|2013-07-25 00:00:...|       9|PENDING_PAYMENT|
|             5648|2013-07-25 00:00:...|      10|PENDING_PAYMENT|
|              918|2013-07-25 00:00:...|      11| PAYMENT_REVIEW|
|             1837|2013-07-25 00:00:...|      12|         CLOSED|
|         

In [23]:
df.printSchema()

root
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- counts: long (nullable = true)



In [24]:
df.select('city').show()

+-----------+
|       city|
+-----------+
|Los Angeles|
|   New York|
|     London|
+-----------+



In [25]:
import pyspark.sql.functions as F

In [26]:
df.select(F.col('country')).show()

+--------------+
|       country|
+--------------+
| United States|
| United States|
|United Kingdom|
+--------------+



In [27]:
df.select('country','city').show()

+--------------+-----------+
|       country|       city|
+--------------+-----------+
| United States|Los Angeles|
| United States|   New York|
|United Kingdom|     London|
+--------------+-----------+



In [28]:
df.select(F.expr('country as destination')).show()

+--------------+
|   destination|
+--------------+
| United States|
| United States|
|United Kingdom|
+--------------+



In [29]:
df.select(F.expr('country as destination').alias('Bhavi_destination')).show()

+-----------------+
|Bhavi_destination|
+-----------------+
|    United States|
|    United States|
|   United Kingdom|
+-----------------+



In [31]:
import pyspark.sql.types as T

In [32]:
rows1 = [
    Row("2020-01-03"),
    Row("2020 01 05"),
    Row("2020 Jan 10"),
    Row("Sat, 11 Jan 2020")
]

In [33]:
myrdd = spark.sparkContext.parallelize(rows)

In [38]:
schema = StructType([
    StructField(name="date_str",dataType=StringType(),nullable=True)
    ])

df = spark.createDataFrame(myrdd,schema)
df

DataFrame[date_str: string]

In [42]:
df = df.withColumn(
    "date",
    F.when(
        F.to_date(F.col('date_str'),"yyyy-MM-dd").isNotNull(),
        F.to_date(F.col('date_str'),"yyyy-MM-dd"),
    ).otherwise(
        F.when(
            F.to_date(F.col('date_str'),"yyyy MM dd").isNotNull(),
            F.to_date(F.col('date_str'),"yyyy MM dd"),
        ).otherwise(
                F.when(
                    F.to_date(F.col('date_str'),"yyyy MMMM dd").isNotNull(),
                    F.to_date(F.col('date_str'),"yyyy MMMM dd"),
            ).otherwise(
                        F.when(
                            F.to_date(F.col('date_str'),"E, dd MMMM yy").isNotNull(),
                            F.to_date(F.col('date_str'),"E, dd MMMM yy"),
                )
            )
        )
    )
)

In [43]:
df.printSchema()

root
 |-- date_str: string (nullable = true)
 |-- timesstamp: timestamp (nullable = true)
 |-- date: date (nullable = true)



In [44]:
df.show()

Py4JJavaError: An error occurred while calling o254.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 22.0 failed 1 times, most recent failure: Lost task 2.0 in stage 22.0 (TID 40, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark-2.4.4-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 377, in main
  File "C:\spark-2.4.4-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 372, in process
  File "C:\spark-2.4.4-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark-2.4.4-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "C:\spark-2.4.4-bin-hadoop2.7\python\pyspark\sql\session.py", line 730, in prepare
    verify_func(obj)
  File "C:\spark-2.4.4-bin-hadoop2.7\python\pyspark\sql\types.py",